In [1]:
%pip install --upgrade --quiet langchain langchain-community langchainhub langchain-openai chromadb bs4 python-dotenv xmltodict pypdf vllm

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
import bs4
import dotenv
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain_community.document_loaders import PubMedLoader

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader

dotenv.load_dotenv()

True

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoModel
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True) 

model_name = "jinaai/jina-embeddings-v2-base-en"
model_kwargs = {'device': 'cpu'}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
)

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.11.intermedi

In [5]:


# load the document and split it into chunks
loader = PyPDFLoader("GBS2.pdf")
docs = loader.load()
len(docs)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("macadeliccc/rag-document-creation")

In [6]:
from langchain_community.llms import VLLM

llm = VLLM(
    model="macadeliccc/laser-dolphin-mixtral-2x7b-dpo-AWQ",
    trust_remote_code=True,
    max_new_tokens=512,
    vllm_kwargs={"quantization": "awq"},
    seed=42,
)


WARNING 02-20 16:16:04 config.py:180] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 02-20 16:16:04 llm_engine.py:79] Initializing an LLM engine with config: model='macadeliccc/laser-dolphin-mixtral-2x7b-dpo-AWQ', tokenizer='macadeliccc/laser-dolphin-mixtral-2x7b-dpo-AWQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)
INFO 02-20 16:16:06 weight_utils.py:163] Using model weights format ['*.safetensors']
INFO 02-20 16:16:12 llm_engine.py:337] # GPU blocks: 5308, # CPU blocks: 2048
INFO 02-20 16:16:13 model_runner.py:666] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=Tru

In [7]:
dr_name="Dr.Kevorkian"
patient_name="Timothy Dolan"
hospital_name="Heidlberg University Hospital"
condition="Guillain-Barré Syndrome (GBS)"
date="2/20/24"
patient_information= """
Clinical Examination

    Vital Signs: BP 125/80 mmHg, HR 88 bpm, Temp 37.2°C, RR 16/min
    General Examination: Alert, oriented, no distress
    Neurological Examination:
        Mental Status: Normal, no cognitive impairment
        Cranial Nerves: Intact
        Motor Function: Weakness observed in bilateral lower extremities, 3/5 strength; upper extremities strength 5/5
        Sensory Examination: Decreased sensation to light touch and pinprick in lower extremities; proprioception intact
        Reflexes: Absent ankle reflexes; reduced knee reflexes
        Coordination: No ataxia noted, but patient has difficulty with heel-to-toe walking due to weakness
    Gait: Unable to walk without assistance, ataxic gait

Laboratory Tests

    Complete Blood Count (CBC): Within normal limits
    Electrolytes: Within normal limits
    Liver Function Tests (LFTs): Within normal limits
    Cerebrospinal Fluid (CSF) Analysis: Albuminocytological dissociation (high protein with normal cell count)
    Nerve Conduction Studies: Slowed conduction velocities and prolonged distal latencies consistent with demyelinating neuropathy

Imaging

    MRI Spine: No spinal cord compression or other abnormalities

Diagnosis

    Primary Diagnosis: Guillain-Barré Syndrome (GBS)
    Secondary Diagnoses: None

Treatment Plan

    Hospitalization in the neurology unit for monitoring and treatment.
    Intravenous Immunoglobulin (IVIG) therapy initiated.
    Physical Therapy consultation for mobility and strength exercises.
    Regular monitoring of respiratory function and swallowing ability to preemptively address potential complications.
    Pain Management as needed for neuropathic pain.

Follow-Up and Prognosis

    Patient to be monitored daily for signs of disease progression, particularly respiratory involvement.
    Expected to begin showing signs of improvement within weeks, with a gradual recovery over several months. Full recovery is possible but may take up to a year or more for some patients.
    Scheduled for follow-up appointments in the outpatient neurology clinic after discharge for ongoing assessment and rehabilitation.
"""

In [8]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
# Example wrapper for static values
def return_value(val):
    # Modified to accept an unused argument
    return lambda _: val


rag_chain = (
    {
    "medical_documents": retriever | format_docs,
    "dr_input": RunnablePassthrough(),
    "doctor_name": return_value(dr_name),
    "patient_name": return_value(patient_name),
    "patient_information": return_value(patient_information),
    "hospital_name": return_value(hospital_name),
    "condition": return_value(condition),
    "date": return_value(date),
    }
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Write the body of the discharge letter based on the patient information and doctor input you have been provided. If you need to you can access medical documents to support your claim.")

Processed prompts: 100%|██████████████████████████| 1/1 [00:05<00:00,  5.91s/it]


'\n\nDear Timothy Dolan,\n\nWe are writing to inform you about your discharge from the Neurology unit at Heidlberg University Hospital following your recent hospitalization. Your thorough evaluation and careful monitoring during your stay have confirmed the primary diagnosis of Guillain-Barré Syndrome (GBS).\n\nDuring your hospitalization, you received much-needed support and treatment, including Intravenous Immunoglobulin (IVIG) therapy, physical therapy, and regular medical evaluations. The primary objectives of your treatment were to alleviate symptoms, prevent complications, and initiate the recovery process.\n\nAs a result of your treatment and care, you have shown signs of improvement in your motor function and overall well-being. However, it is essential to continue with ongoing follow-ups in the outpatient neurology clinic for regular assessments and support in your rehabilitation journey.\n\nWe have scheduled follow-up appointments for you, ensuring our multidisciplinary team 

In [9]:
# cleanup
vectorstore.delete_collection()